In [1]:
using Tensors

In [2]:
function neo_hooke_energy(C::SymmetricTensor{2,3}, μ, K)
    I1 = tr(C)
    J = sqrt(det(C))
    return μ / 2 * (I1 / J^(2 / 3) - 3) + K / 2 * (log(J))^2
end


neo_hooke_energy (generic function with 1 method)

In [3]:
function PK2_stress(C::SymmetricTensor{2,3}, μ, K)
    return 2 * gradient(C -> neo_hooke_energy(C, μ, K), C)
end


PK2_stress (generic function with 1 method)

In [4]:
function material_tangent(C::SymmetricTensor{2,3}, μ, K)
    return 4 * hessian(C -> neo_hooke_energy(C, μ, K), C)
end


material_tangent (generic function with 1 method)

In [5]:
μ = 80.0
K = 150.0

# deformációgradiens (pl. nyírás + nyúlás)
F = Tensor{2,3}((1.2, 0.3, 0.0,
    0.0, 1.1, 0.0,
    0.0, 0.0, 1.0))

C = symmetric(F' ⋅ F)

φ = neo_hooke_energy(C, μ, K)
S = PK2_stress(C, μ, K)
𝓒 = material_tangent(C, μ, K)

println("Energy φ = ", φ)
println("PK2 stress S = ")
println(S)
println("Material tangent 𝓒 = ")
println(𝓒)


Energy φ = 10.103235442632588
PK2 stress S = 
[37.84586425819877 7.809992565723037 0.0; 7.809992565723037 30.27253813386129 0.0; 0.0 0.0 25.24574291883227]
Material tangent 𝓒 = 
[77.19977910241924 -29.448740723115584 0.0; -29.448740723115584 58.42284546590683 0.0; 0.0 0.0 67.4371353204101;;; -29.44874072311559 33.98746129395842 0.0; 33.98746129395842 -39.45680584350711 0.0; 0.0 0.0 -30.47967393572093;;; 0.0 0.0 28.63663940765115; 0.0 0.0 -7.809992565723037; 28.63663940765115 -7.809992565723037 0.0;;;; -29.44874072311559 33.98746129395842 0.0; 33.98746129395842 -39.45680584350711 0.0; 0.0 0.0 -30.47967393572093;;; 58.422845465906825 -39.456805843507105 0.0; -39.456805843507105 144.01727880338666 0.0; 0.0 0.0 96.99318277323039;;; 0.0 0.0 -7.809992565723037; 0.0 0.0 36.20996553198863; -7.809992565723037 36.20996553198863 0.0;;;; 0.0 0.0 28.63663940765115; 0.0 0.0 -7.809992565723037; 28.63663940765115 -7.809992565723037 0.0;;; 0.0 0.0 -7.809992565723037; 0.0 0.0 36.20996553198863; -7.80999

In [6]:
C_voigt = tomandel(𝓒)


6×6 Matrix{Float64}:
  77.1998   58.4228   67.4371    0.0       0.0     -41.6468
  58.4228  144.017    96.9932    0.0       0.0     -55.8003
  67.4371   96.9932  199.085     0.0       0.0     -43.1048
   0.0       0.0       0.0      72.4199  -15.62      0.0
   0.0       0.0       0.0     -15.62     57.2733    0.0
 -41.6468  -55.8003  -43.1048    0.0       0.0      67.9749

In [7]:
C_voigt = tovoigt(𝓒)


6×6 Matrix{Float64}:
  77.1998   58.4228   67.4371   0.0       0.0      -29.4487
  58.4228  144.017    96.9932   0.0       0.0      -39.4568
  67.4371   96.9932  199.085    0.0       0.0      -30.4797
   0.0       0.0       0.0     36.21     -7.80999    0.0
   0.0       0.0       0.0     -7.80999  28.6366     0.0
 -29.4487  -39.4568  -30.4797   0.0       0.0       33.9875

In [8]:
using Tensors

μ = 80.0
K = 150.0

F = Tensor{2,3}((1.2, 0.3, 0.0,
    0.0, 1.1, 0.0,
    0.0, 0.0, 1.0))

C = symmetric(F' ⋅ F)

function neo_hooke_energy(C::SymmetricTensor{2,3}, μ, K)
    I1 = tr(C)
    J = sqrt(det(C))
    return μ / 2 * (I1 / J^(2 / 3) - 3) + K / 2 * (log(J))^2
end

S = 2 * gradient(C -> neo_hooke_energy(C, μ, K), C)
𝓒 = 4 * hessian(C -> neo_hooke_energy(C, μ, K), C)

S_voigt = tomandel(S)    # 6×1
C_voigt = tomandel(𝓒)   # 6×6

println("S_voigt = ", S_voigt)
println("C_voigt = ")
println(C_voigt)


S_voigt = [37.84586425819877, 30.27253813386129, 25.24574291883227, 0.0, 0.0, 11.044997408478565]
C_voigt = 
[77.19977910241924 58.422845465906825 67.4371353204101 0.0 0.0 -41.646808525438935; 58.42284546590683 144.01727880338666 96.99318277323042 0.0 0.0 -55.80034995180975; 67.4371353204101 96.99318277323039 199.08453076407517 0.0 0.0 -43.10476825660627; 0.0 0.0 0.0 72.41993106397727 -15.619985131446077 0.0; 0.0 0.0 0.0 -15.619985131446077 57.27327881530231 0.0; -41.64680852543893 -55.80034995180974 -43.10476825660626 0.0 0.0 67.97492258791685]


A tomandel:

- energiakonzisztens

- belső munka:

𝜎:𝜀=𝜎𝑇𝜀
σ:ε=σTε
	​

Newton-tangens helyes marad

In [1]:
using LowLevelFEM
#using Tensors

LowLevelFEM.tensor_test()

LoadError: tensor_extension_test requires the Tensors backend.

Please run first:
    using Tensors
